In [1]:
import numpy as np
import pickle
#State: Oberservation
#Argument:
#     data : 紀錄3*3的圈圈叉叉棋盤
#     winner : winner=1圈贏，winner=-1叉贏 winner=0平手
#     end : end=False未結束，end=True結束
#     hash value : 湊雜值，可以理解成每一個state的指紋，其是獨一無二的
#function:
#    is_end : 檢查遊戲結束與否
#    next_move : 輸入哪一方下哪裡，回傳下完後的state
#    print_state : 印出棋盤
#    compute_hash : 計算湊雜值
class State:
    def __init__(self):
        self.data = np.zeros((3,3))
        self.winner = None
        self.end = None
        self.hash_val = None
    def is_end(self):
        if self.end == None:
            all_sum = []
            #check rows
            for i in np.sum(self.data, axis = 1):
                all_sum.append(i)
            #check columns
            for i in np.sum(self.data, axis = 0):
                all_sum.append(i)
            #check diagonals
            all_sum.append(self.data[0,0]+self.data[1,1]+self.data[2,0])
            all_sum.append(self.data[0,2]+self.data[1,1]+self.data[2,2])
            
            for i in range(len(all_sum)):
                if all_sum[i] == 3:
                    self.winner = 1
                    self.end = True
                    return self.end
                elif all_sum[i] == -3:
                    self.winner = -1
                    self.end = True
                    return self.end
            
            #檢查平手
            is_full=1
            for i in range(self.data.shape[0]):
                for j in range(self.data.shape[1]):
                    if  self.data[i, j] == 0:
                        is_full=0
            if is_full == 1:
                self.winner = 0
                self.end = True
                return self.end
            
            # 上述情形都沒有發生
            self.end = False
            return self.end
        else:
            return self.end
        
    def next_move(self, i, j, symbol):
        #symbol代表你是1方還是-1方
        #i,j代表你要下在哪裡
        next_state = State()
        next_state.data = np.copy(self.data) #傳值不傳址
        next_state.data[i, j] = symbol
        return next_state
    
    
    def print_state(self):
        for i in range(self.data.shape[0]):
            print('-------------')
            out = '| '
            for j in range(self.data.shape[1]):
                if self.data[i, j] == 1:
                    token = '*'
                elif self.data[i, j] == -1:
                    token = 'x'
                else:
                    token = '0'
                out += token + ' | '
            print(out)
        print('-------------')
    
    def compute_hash(self):
        if self.hash_val is None:
            self.hash_val = 0
            for i in np.nditer(self.data):
                self.hash_val = self.hash_val * 3 + i + 1
        return self.hash_val
    
#以current_state為起點，往下取得所有可能的state，並將其放入all_state這個字典
#current_symbol:現在輪到哪一方下
def get_all_state_impl(current_state,current_symbol,all_state):
    for i in range(current_state.data.shape[0]):
        for j in range(current_state.data.shape[1]):
            if current_state.data[i,j]==0:
                next_state=current_state.next_move(i, j, current_symbol)
                next_state_hash=next_state.compute_hash()
                if next_state_hash not in all_state:
                    all_state[next_state_hash]=(next_state,next_state.is_end())
                if next_state.is_end()==False:
                    get_all_state_impl(next_state,-current_symbol,all_state)
    
#取得所有State
def get_all_states():
    all_state={}
    #先手的一定是1
    current_symbol=1
    current_state=State()
    all_state[current_state.compute_hash()] = (current_state, current_state.is_end())
    get_all_state_impl(current_state,current_symbol,all_state)
    return all_state   

In [2]:
all_states = get_all_states()

In [3]:
#總共有5488種state
print(len(all_states))

5488


In [27]:
#Judger: 掌管遊戲的進行
#input:
#     player1 : 先攻方(class Player)
#     player2 : 後攻方(class Player)
#Argument:
#     p1 : 等於player1
#     p2 : 等於player2
#     p1_symbol : 先攻方以1代表，一開始即呼叫set_symbol賦予player1 symbol
#     p2_symbol : 先攻方以-1代表，一開始即呼叫set_symbol賦予player2 symbol
#     current_sate : 現在的棋盤
#Function:
#     reset : 清空player1和player2的states和greedy
#     alternate : 回傳現在輪到哪一方
#     play : 進行遊戲直到end=true
class Judger:
    def __init__(self,player1,player2):
        self.p1 = player1
        self.p2 = player2
        self.current_player = None
        self.p1_symbol = 1
        self.p2_symbol = -1
        self.p1.set_symbol(self.p1_symbol)
        self.p2.set_symbol(self.p2_symbol)
        self.current_state = State()
        
    def reset(self):
        self.p1.reset()
        self.p2.reset()
        
    def alternate(self):
        while True:
            yield self.p1
            yield self.p2
    
    def play(self, print_state=False):
        alternator = self.alternate()
        self.reset()
        current_state = State()
        self.p1.set_state(current_state)
        self.p2.set_state(current_state)
        if print_state:
            current_state.print_state()
        while True:
            #yield需要next才能進行，視作return，當在下一次遇到next時，執行下一行
            player = next(alternator)
            i, j, symbol = player.act()
            next_state_hash = current_state.next_move(i, j, symbol).compute_hash()
            current_state, is_end = all_states[next_state_hash]
            self.p1.set_state(current_state)
            self.p2.set_state(current_state)
            if print_state:
                current_state.print_state()
            #無限迴圈直到分出勝負
            if is_end:
                return current_state.winner
#Player: 電腦玩家
#input:
#     step_size : learning rate
#     epsilon: the probability to explore，即隨機選擇要下的位置，探索沒看過的點，而不是挑選value值最大的位置
#Argument:
#     estimations : 儲存每一個state的value值
#     states : 一局遊戲到目前為止所有的state
#     greedy : 下一步是exploration(false)還是greedy(true)
#Function:
#     reset : 清空player的states和greedy
#     set_state : 更新新的state到states，greedy也一同更新，直接賦予其true
#     set_symbol : 給予所有state初始value值，依據你現在是哪一方而不同
#                  if win(value=1) ,lose(value=0) ,tie and not end(value=0.5)
#     back_up : 當一局遊戲結束時，向後更新所有的value值
#     act : 下一步要下哪裡，有兩種策略(greedy and exploration)
#     save_policy : 訓練結束時，將estimations儲存起來
#     load_plicy : 載入訓練好的estimations，做greedy用
class Player:
    def __init__(self, step_size=0.1, epsilon=0.1):
        self.estimations = {}
        self.step_size = step_size
        self.epsilon = epsilon
        self.states = []
        self.greedy = []
        self.symbol = 0
        
    def reset(self):
        self.states = []
        self.greedy = []
        
    def set_state(self, state):
        self.states.append(state)
        self.greedy.append(True)  

    def set_symbol(self, symbol):
        self.symbol = symbol
        for hash_val in all_states:
            #將all_sates裡的存的class State和is_end取出
            state, is_end = all_states[hash_val]
            if is_end == True:
                if self.symbol == state.winner:
                    self.estimations[hash_val] = 1.0
                elif -self.symbol == state.winner:
                    self.estimations[hash_val] = 0
                else:
                    self.estimations[hash_val] = 0.5
            else:
                self.estimations[hash_val] = 0.5
                
    def backup(self):
        states_hash = [state.compute_hash() for state in self.states]
        #遊戲結束的最後一張state，value已經決定
        for i in reversed(range(len(states_hash) - 1)):
            state_hash = states_hash[i]
            #探索move不會導致任何學習
            td_error = self.greedy[i] * (
                self.estimations[states_hash[i + 1]] - self.estimations[state_hash]
            )
            self.estimations[state_hash] += td_error
            
    def act(self):
        #現在的棋盤
        state = self.states[-1]
        next_states = []
        #所有可能下的位置
        next_positions = []
        for i in range(state.data.shape[0]):
            for j in range(state.data.shape[1]):
                if state.data[i,j] == 0:
                    next_positions.append([i,j])
                    next_state = state.next_move(i, j, self.symbol)
                    next_states.append(next_state.compute_hash())
        
        #如果機率落在探索，隨機選擇一個點
        #np.random.rand()，返回0到1之間的隨機值，np.random.randint(i)，返回"小於"i的隨機值
        if np.random.rand() < self.epsilon:
            action = next_positions[np.random.randint(len(next_positions))]
            action.append(self.symbol)
            self.greedy[-1] = False
            return action
        #values:[value值,位置]
        values = []
        for i in range(len(next_states)):
            values.append((self.estimations[next_states[i]],next_positions[i]))
        np.random.shuffle(values)
        #透過values的第0個元素排序
        values.sort(key=lambda x: x[0], reverse=True)
        action = values[0][1]
        action.append(self.symbol)
        return action
    
    def save_policy(self):
        if self.symbol == 1:
            f_or_s='first'
        else:
            f_or_s='second'
        
        with open('policy_'+f_or_s+'.bin', 'wb') as f:
            pickle.dump(self.estimations, f)

    def load_policy(self):
        if self.symbol == 1:
            f_or_s='first'
        else:
            f_or_s='second'
        
        with open('policy_'+f_or_s+'.bin', 'rb') as f:
            self.estimations = pickle.load(f)


In [20]:
def train(epochs, print_every_n=10000):
    player1 = Player(epsilon=0.01)
    player2 = Player(epsilon=0.01)
    judger = Judger(player1, player2)
    player1_win = 0.0
    player2_win = 0.0
    for i in range(1, epochs + 1):
        winner = judger.play(print_state=False)
        if winner == 1:
            player1_win += 1
        if winner == -1:
            player2_win += 1
        if i % print_every_n == 0:
            print('Epoch %d, player 1 winrate: %.02f, player 2 winrate: %.02f' % (i, player1_win / i, player2_win / i))
        player1.backup()
        player2.backup()
        judger.reset()
    player1.save_policy()
    player2.save_policy()


def compete(turns):
    #不再探索
    player1 = Player(epsilon=0)
    player2 = Player(epsilon=0)
    judger = Judger(player1, player2)
    player1.load_policy()
    player2.load_policy()
    player1_win = 0.0
    player2_win = 0.0
    for _ in range(turns):
        winner = judger.play()
        if winner == 1:
            player1_win += 1
        if winner == -1:
            player2_win += 1
        judger.reset()
    print('%d turns, player 1 win %.02f, player 2 win %.02f' % (turns, player1_win / turns, player2_win / turns))


In [28]:
#訓練10^5次
train(int(1e5))
#對戰10^3，如果value值正確，應該都是平手
compete(int(1e3))

Epoch 10000, player 1 winrate: 0.32, player 2 winrate: 0.05
Epoch 20000, player 1 winrate: 0.22, player 2 winrate: 0.03
Epoch 30000, player 1 winrate: 0.16, player 2 winrate: 0.03
Epoch 40000, player 1 winrate: 0.13, player 2 winrate: 0.02
Epoch 50000, player 1 winrate: 0.11, player 2 winrate: 0.02
Epoch 60000, player 1 winrate: 0.09, player 2 winrate: 0.02
Epoch 70000, player 1 winrate: 0.08, player 2 winrate: 0.02
Epoch 80000, player 1 winrate: 0.08, player 2 winrate: 0.02
Epoch 90000, player 1 winrate: 0.07, player 2 winrate: 0.02
Epoch 100000, player 1 winrate: 0.07, player 2 winrate: 0.02
1000 turns, player 1 win 0.00, player 2 win 0.00


In [48]:
class human_player():
    def __init__(self, **kwargs):
        self.symbol = None
        self.keys = [1,2,3,4,5,6,7,8,9]
        self.state = None
    def reset(self):
        pass

    def set_state(self, state):
        self.state = state

    def set_symbol(self, symbol):
        self.symbol = symbol
    
    def act(self):
        self.state.print_state()
        key = input("Input your position:")
        key=int(key)
        i = key // 3
        if key % 3 == 0:
            i-=1
        i = 2-i
        j = key % 3 -1
        if j<0:
            j=2
        return i, j, self.symbol

def play():
    key = np.array([[7,8,9],[4,5,6],[1,2,3]])
    print('position')
    for i in range(key.shape[0]):
            print('-------------')
            out = '| '
            for j in range(key.shape[1]):
                out += str(key[i,j]) + ' | '
            print(out)
    print('-------------')
    while True:
        player1 = human_player()
        player2 = Player(epsilon=0)
        judger = Judger(player1, player2)
        player2.load_policy()
        winner = judger.play()
        if winner == player2.symbol:
            print("You lose!")
        elif winner == player1.symbol:
            print("You win!")
        else:
            print("It is a tie!")
        
    

In [ ]:
play()